#video classification (1) of COVID-US (data preparation)

https://www.analyticsvidhya.com/blog/2019/09/step-by-step-deep-learning-tutorial-video-classification-python/

**preprocessed(clean) videos -> split -> pre-trained CNN** -> LSTM or GRU -> dense layer -> result(accuracy)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/dp/

/content/drive/MyDrive/dp


In [3]:
route = '/content/drive/MyDrive/dp/'

##0. Import libraries

In [4]:
import cv2 # for capturing videos
import math # for mathematical operations
%matplotlib inline
import matplotlib.pyplot as plt # for plotting the images
import numpy as np # for mathematical operations
import pandas as pd
import tensorflow as tf

from skimage.transform import resize # for resizing images
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm

import keras
from keras.utils import np_utils
from keras.preprocessing import image # for preprocessing the images
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.layers import TimeDistributed, GRU, LSTM, GlobalAveragePooling1D

from scipy import stats as s
from sklearn.metrics import accuracy_score

import os
import time
import random
import PIL
from PIL import Image
# import re
# import shutil

In [5]:
frames=14
width=224
height=224
channels=3
train_test_ratio=0.3
train_val_ratio=0.2

##1. Prepare preprocessd(clean) videos

In [6]:
video_dataset = pd.read_csv("utils/video_files_clean_properties_addition.csv")
video_dataset

,filename,framerate,width,height,frame_count,duration_secs,disease,source
0,1_butterfly_covid_prc_convex_clean.avi,15.0,820.0,820.0,64.0,4.27,0,1
1,2_butterfly_covid_prc_convex_clean.avi,15.0,624.0,624.0,158.0,10.53,0,1
2,3_butterfly_covid_prc_convex_clean.avi,15.0,1054.0,1054.0,90.0,6.00,0,1
3,4_butterfly_covid_prc_convex_clean.avi,15.0,820.0,820.0,108.0,7.20,0,1
4,5_butterfly_covid_prc_convex_clean.avi,15.0,810.0,810.0,249.0,16.60,0,1
...,...,...,...,...,...,...,...,...
237,239_paper_normal_prc_convex_clean.avi,15.0,512.0,512.0,197.0,13.13,2,8
238,240_paper_other_prc_convex_clean.avi,15.0,512.0,512.0,96.0,6.40,2,8
239,241_paper_other_prc_convex_clean.avi,15.0,512.0,512.0,98.0,6.53,2,8
240,242_paper_other_prc_convex_clean.avi,15.0,512.0,512.0,299.0,19.93,2,8


In [7]:
video_dataset=video_dataset[video_dataset['filename'].str.contains('covid|pneumonia|normal')]
video_dataset=video_dataset[video_dataset['filename'].str.contains('convex')]

video_dataset

,filename,framerate,width,height,frame_count,duration_secs,disease,source
0,1_butterfly_covid_prc_convex_clean.avi,15.0,820.0,820.0,64.0,4.27,0,1
1,2_butterfly_covid_prc_convex_clean.avi,15.0,624.0,624.0,158.0,10.53,0,1
2,3_butterfly_covid_prc_convex_clean.avi,15.0,1054.0,1054.0,90.0,6.00,0,1
3,4_butterfly_covid_prc_convex_clean.avi,15.0,820.0,820.0,108.0,7.20,0,1
4,5_butterfly_covid_prc_convex_clean.avi,15.0,810.0,810.0,249.0,16.60,0,1
...,...,...,...,...,...,...,...,...
231,233_clarius_pneumonia_prc_convex_clean.avi,15.0,458.0,458.0,482.0,32.13,1,9
232,234_clarius_pneumonia_prc_convex_clean.avi,15.0,458.0,458.0,482.0,32.13,1,9
233,235_clarius_pneumonia_prc_convex_clean.avi,15.0,458.0,458.0,481.0,32.07,1,9
234,236_clarius_normal_prc_convex_clean.avi,15.0,482.0,482.0,365.0,24.33,2,9


In [8]:
# separating the target
X = video_dataset
y = video_dataset['disease']

# creating the training and validation set
X_training, X_test, y_training, y_test = train_test_split(X, y, test_size=train_test_ratio, stratify = y)
X_train, X_val, y_train, y_val = train_test_split(X_training, y_training, shuffle=True, test_size=train_val_ratio, stratify = y_training)

X_train = X_train.reset_index(drop = True)
X_val = X_val.reset_index(drop = True)
X_test = X_test.reset_index(drop = True)
y_train = y_train.reset_index(drop = True)
y_val = y_val.reset_index(drop = True)
y_test = y_test.reset_index(drop = True)

In [9]:
print("X_train size : {}".format(X_train.shape))
print("y_train size : {}".format(y_train.shape))
print("X_val size : {}".format(X_val.shape))
print("y_val size : {}".format(y_val.shape))
print("X_test size : {}".format(X_test.shape))
print("y_test size : {}".format(y_test.shape))

X_train size : (68, 8)
y_train size : (68,)
X_val size : (17, 8)
y_val size : (17,)
X_test size : (37, 8)
y_test size : (37,)


test 영상만 이후 evaluate 하는 과정에서 frame을 추출하므로 파일 이름을 csv 파일로 저장

In [10]:
# storing the frames from training videos
test_dataset = pd.DataFrame()
test_dataset['filename'] = X_test['filename']
test_dataset.to_csv('data/test_dataset.csv', header=True, index=False)

train 영상과 val 영상은 지금 frame을 추출

다음 과정에서 생성되는 폴더는 코드를 다시 실행하는 경우 중복이나 필요없는 데이터를 제거하기 위해 이 폴더를 삭제하고 진행 (다음 코드 참고)

In [11]:
# create images_14frames_train and images_14frames_val folders, if they do not exist
if not os.path.exists('data/images_captured_train'):
    os.makedirs('data/images_captured_train')
if not os.path.exists('data/images_captured_val'):
    os.makedirs('data/images_captured_val')

files = glob('data/images_captured_train/*')
for f in files:
    os.remove(f)
files = glob('data/images_captured_val/*')
for f in files:
    os.remove(f)

Converts a PIL Image instance to a Numpy array. 
(tf.keras.image.img_to_array와 동일)

In [12]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
	  (im_height, im_width, channels)).astype(np.uint8)
   
def featuring_space(frame_count):
  if frame_count<28:
    return 1
  elif frame_count<56:
    return 2
  elif frame_count<80:
    return 3
  elif frame_count<120:
    return 4
  else : return 5

**모든 영상에서 뽑아내는 frame 길이를 동일하게 맞춘 뒤에 LSTM을 통과시키고자**

**모든 영상에 대해 frames에 해당하는 프레임을 capture함**

In [13]:
train_label=[]
for i in tqdm(range(X_train.shape[0])):

    count = 0
    current = 0
    videoFile = X_train['filename'][i]
    disease = videoFile.split('_')[2]
    if disease == 'covid' :
      disease_num = 0
    elif disease == 'pneumonia' :
      disease_num = 1
    else : disease_num = 2
    cap = cv2.VideoCapture('data/video/clean/' + videoFile) # capturing the video from the given path
    frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    space = featuring_space(frame_count)
    capture_epoch = math.floor( frame_count / (14*space) )
    capture_num = capture_epoch*14

    while(cap.isOpened()):
      if(count==capture_num):
        break
      cap.get(1)
      ret, frame = cap.read()
      if (ret != True):
        break
      if current%space==0 :
        filename = 'data/images_captured_train/' + videoFile + "_frame_%d.jpg" % count; count += 1
        cv2.imwrite(filename, frame)
        if(count%14==0):
          train_label.append(disease_num)
      current=current+1
    cap.release()

len(os.listdir('data/images_captured_train/'))

100%|██████████| 68/68 [00:41<00:00,  1.65it/s]


1722

In [14]:
val_label=[]
for i in tqdm(range(X_val.shape[0])):

    count = 0
    current = 0
    videoFile = X_val['filename'][i]
    disease = videoFile.split('_')[2]
    if disease == 'covid' :
      disease_num = 0
    elif disease == 'pneumonia' :
      disease_num = 1
    else : disease_num = 2
    cap = cv2.VideoCapture('data/video/clean/' + videoFile) # capturing the video from the given path
    frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    space = featuring_space(frame_count)
    capture_epoch = math.floor( frame_count / (14*space) )
    capture_num = capture_epoch*14

    while(cap.isOpened()):
      if(count==capture_num):
        break
      cap.get(1)
      ret, frame = cap.read()
      if (ret != True):
        break
      if current%space==0 :
        filename = 'data/images_captured_val/' + videoFile + "_frame_%d.jpg" % count; count += 1
        cv2.imwrite(filename, frame)
        if(count%14==0):
          val_label.append(disease_num)
      current=current+1
    cap.release()

len(os.listdir('data/images_captured_val/'))

100%|██████████| 17/17 [00:10<00:00,  1.66it/s]


406

In [15]:
X_val

,filename,framerate,width,height,frame_count,duration_secs,disease,source
0,220_clarius_covid_prc_convex_clean.avi,15.0,400.0,400.0,118.0,7.87,0,9
1,12_butterfly_covid_prc_convex_clean.avi,15.0,820.0,820.0,111.0,7.40,0,1
2,203_paper_covid_prc_convex_clean.avi,15.0,680.0,680.0,242.0,16.13,0,8
3,191_uf_normal_prc_convex_clean.avi,15.0,542.0,542.0,120.0,8.00,2,7
4,85_butterfly_covid_prc_convex_clean.avi,15.0,622.0,622.0,153.0,10.20,0,1
5,9_butterfly_covid_prc_convex_clean.avi,15.0,1054.0,1054.0,80.0,5.33,0,1
6,81_butterfly_covid_prc_convex_clean.avi,15.0,656.0,656.0,242.0,16.13,0,1
7,73_pocusatlas_covid_prc_convex_clean.avi,15.0,406.0,406.0,46.0,3.07,0,4
8,80_pocusatlas_pneumonia_prc_convex_clean.avi,15.0,196.0,196.0,30.0,2.00,1,4
9,221_clarius_covid_prc_convex_clean.avi,15.0,400.0,400.0,83.0,5.53,0,9


In [16]:
train_images = glob("data/images_captured_train/*.jpg")

train_image_name = []
train_image_class = []
for i in tqdm(range(len(train_images))):
    # creating the image name
    train_image_name.append(train_images[i].split('/')[2])
    # creating the class of image
    disease = train_images[i].split('/')[2].split('_')[2]
    if disease == 'covid':
        train_image_class.append(0)
    elif disease == 'pneumonia':
      train_image_class.append(1)
    else:
        train_image_class.append(2)

train_image_dataset = pd.DataFrame()
train_image_dataset['filename'] = train_image_name
train_image_dataset['disease'] = train_image_class # 

train_image_dataset.to_csv('data/image_files_clean_train.csv', header=True, index=False)

100%|██████████| 1722/1722 [00:00<00:00, 620018.16it/s]


In [17]:
train_image_dataset

,filename,disease
0,87_butterfly_covid_prc_convex_clean.avi_frame_...,0
1,87_butterfly_covid_prc_convex_clean.avi_frame_...,0
2,87_butterfly_covid_prc_convex_clean.avi_frame_...,0
3,87_butterfly_covid_prc_convex_clean.avi_frame_...,0
4,87_butterfly_covid_prc_convex_clean.avi_frame_...,0
...,...,...
1717,44_litfl_pneumonia_prc_convex_clean.avi_frame_...,1
1718,44_litfl_pneumonia_prc_convex_clean.avi_frame_...,1
1719,44_litfl_pneumonia_prc_convex_clean.avi_frame_...,1
1720,44_litfl_pneumonia_prc_convex_clean.avi_frame_...,1


In [18]:
val_images = glob("data/images_captured_val/*.jpg")

val_image_name = []
val_image_class = []
for i in tqdm(range(len(val_images))):
    # creating the image name
    val_image_name.append(val_images[i].split('/')[2])
    # creating the class of image
    disease = val_images[i].split('/')[2].split('_')[2]
    if disease == 'covid':
        val_image_class.append(0)
    elif disease == 'pneumonia':
      val_image_class.append(1)
    else:
        val_image_class.append(2)

val_image_dataset = pd.DataFrame()
val_image_dataset['filename'] = val_image_name
val_image_dataset['disease'] = val_image_class 

val_image_dataset.to_csv('data/image_files_clean_val.csv', header=True, index=False)

100%|██████████| 406/406 [00:00<00:00, 454345.63it/s]


In [19]:
val_image_dataset

,filename,disease
0,220_clarius_covid_prc_convex_clean.avi_frame_0...,0
1,220_clarius_covid_prc_convex_clean.avi_frame_1...,0
2,220_clarius_covid_prc_convex_clean.avi_frame_2...,0
3,220_clarius_covid_prc_convex_clean.avi_frame_3...,0
4,220_clarius_covid_prc_convex_clean.avi_frame_4...,0
...,...,...
401,229_clarius_pneumonia_prc_convex_clean.avi_fra...,1
402,229_clarius_pneumonia_prc_convex_clean.avi_fra...,1
403,229_clarius_pneumonia_prc_convex_clean.avi_fra...,1
404,229_clarius_pneumonia_prc_convex_clean.avi_fra...,1


In [20]:
# creating an empty list
train_images = []

# for loop to read and store frames
for i in tqdm(range(train_image_dataset.shape[0])):
    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img('data/images_captured_train/'+train_image_dataset['filename'][i], target_size=(height,width,channels))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img/255
    # appending the image to the images list
    train_images.append(img)
    
# converting the list to numpy array
X_train = np.array(train_images)
print(X_train.shape)

100%|██████████| 1722/1722 [00:11<00:00, 147.91it/s]


(1722, 224, 224, 3)


In [21]:
# creating an empty list
val_images = []

# for loop to read and store frames
for i in tqdm(range(val_image_dataset.shape[0])):
    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img('data/images_captured_val/'+val_image_dataset['filename'][i], target_size=(height,width,channels))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img/255
    # appending the image to the images list
    val_images.append(img)
    
# converting the list to numpy array
X_val = np.array(val_images)
print(X_val.shape)

100%|██████████| 406/406 [00:02<00:00, 142.92it/s]


(406, 224, 224, 3)


In [24]:
X_train=X_train.reshape(-1,14,224,224,3)
X_val=X_val.reshape(-1,14,224,224,3)
train_label = np.array(train_label)
val_label = np.array(val_label)
print(X_train.shape)
print(X_val.shape)
print(train_label.shape)
print(val_label.shape)
print(X_test.shape)

(123, 14, 224, 224, 3)
(29, 14, 224, 224, 3)
(123,)
(29,)
(37, 8)


In [23]:
np.save('data/X_train_save_0', X_train)
np.save('data/X_val_save_0', X_val)
np.save('data/y_train_save_0', train_label)
np.save('data/y_val_save_0', val_label)